This code was written in a hurry and shoudl be cleaned. It is only shared because some of the competitors asked for it.  It hsould be refactored if anyone intend to reuse it.

In [ ]:
fname = 'final'

Some imports

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from tqdm import tqdm_notebook
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label, binary_erosion, binary_dilation, disk
from skimage.morphology import square, watershed, closing, binary_closing
from skimage.morphology import remove_small_holes, remove_small_objects
from skimage.filters.rank import gradient
from skimage.filters import sobel, scharr, prewitt, roberts
from skimage.exposure import rescale_intensity
from skimage.segmentation import random_walker
from skimage.util import pad

from scipy.special import expit, logit

#import cv2

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

from scipy.ndimage.morphology import binary_fill_holes

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Add, Multiply, ZeroPadding2D
from keras.layers.core import Dropout, Lambda, Dense
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.pooling import MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.merge import concatenate, Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras import backend as K
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.engine import get_source_inputs
from keras.utils import get_file, Sequence

import keras
from distutils.version import StrictVersion

if StrictVersion(keras.__version__) < StrictVersion('2.2.0'):
    from keras.applications.imagenet_utils import _obtain_input_shape
else:
    from keras_applications.imagenet_utils import _obtain_input_shape

#from keras_util import ExponentialMovingAverage
from segmentation_models import Unet

import tensorflow as tf

import pickle as pkl
import gc

# Set some parameters
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
TRAIN_PATH = '../input/train/'
TEST_PATH = '../input/test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

Recommended way to remove noin determinism.  In addiiton, PYTHON_HASH_SEED was set to 0 before running notebooks.  Yet, repeated runs produce different outputs.   

In [ ]:
import random as rn
def init_seeds(seed):

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.

    np.random.seed(seed)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.

    rn.seed(seed)

    # Force TensorFlow to use single thread.
    # Multiple threads are a potential source of
    # non-reproducible results.
    # For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    from keras import backend as K

    # The below tf.set_random_seed() will make random number generation
    # in the TensorFlow backend have a well-defined initial state.
    # For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

    tf.set_random_seed(seed)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    return sess

sess = init_seeds(0)

Reading data

In [ ]:
train_df = pd.read_csv("../input/train.csv", index_col="id", usecols=[0])


depths_df = pd.read_csv("../input/depths.csv", index_col="id")


train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

len(train_df)

train_ids = train_df.index
test_ids = test_df.index

In [ ]:
train = []
train_mask = []

for n, id_ in tqdm_notebook(enumerate(train_ids), total=len(train_ids)):
    file = "../input/train/images/{}.png".format(id_)
    mfile = "../input/train/masks/{}.png".format(id_)
    image = (imread(file) / 255).astype('float32')
    #image = rescale_intensity(image, out_range=np.uint8).astype('float32') / 255
    mask = imread(mfile, as_gray=True) / 65535
    train.append(image)
    train_mask.append(mask)

  

In [ ]:
num_channels = 3
img_size = 101
new_img_size = 256

In [ ]:
train = np.array(train)

In [ ]:
train_mask = np.array(train_mask)

Helper functions for training time augmentation, and image resizing.

In [ ]:
def cov_to_class(img):
    val = np.sum(img) / (img.shape[0] * img.shape[1])
    for i in range(0, 11):
        if val * 10 <= i :
            return i
coverage = [cov_to_class(img) for img in train_mask]
coverage[:5]

In [ ]:
def vertical_mask(mask):
    res = np.sum(mask, axis=0)
    res = np.unique(res)
    return len(res) == 2 and res[0] == 0 and res[1] == mask.shape[0]

In [ ]:
def tta(x, y, incr_sz=30, img_size=img_size):
    if np.random.random()  < 0.33:
        return x, y
    incr = np.random.randint(incr_sz)
    new_sz = incr_sz + img_size
    x = resize(x, (new_sz, img_size), preserve_range=True, order=3)
    y = resize(y, (new_sz, img_size), preserve_range=True, order=3)
    y = 1 * (y > 0.5)
    offset = np.random.randint(incr+1)
    x = x[offset:offset+img_size]
    y = y[offset:offset+img_size]
    return x,y

In [ ]:
def my_resize(img, is_mask, new_img_size=new_img_size):
    if is_mask:
        img = img.reshape((img.shape[0], img.shape[1], 1))
    if new_img_size == 128:
        img = pad(img, ((13, 14), (13, 14), (0, 0)), mode='reflect')
    if new_img_size == 224:
        img = resize(img, (202, 202), preserve_range=True, order=3)
        img = pad(img, ((11, 11), (11, 11), (0, 0)), mode='reflect')
    if new_img_size == 256:
        img = resize(img, (202, 202), preserve_range=True, order=3)
        img = pad(img, ((27, 27), (27, 27), (0, 0)), mode='reflect')
    if is_mask:
        img = (img > 0.5).astype('float32')
    return img

In [ ]:
def my_crop(img, new_img_size=new_img_size):
    num_channel = 0 if len(img.shape) == 2 else img.shape[2]
    if new_img_size == 128:
        img = img[13:13+101, 13:13+101]
    if new_img_size == 224:
        img = img[11:11+202, 11:11+202]
        if num_channel:
            img = resize(img, (101, 101, num_channel), preserve_range=True, order=3)
        else:
            img = resize(img, (101, 101), preserve_range=True, order=3)
    if new_img_size == 256:
        img = img[27:27+202, 27:27+202]
        if num_channel:
            img = resize(img, (101, 101, num_channel), preserve_range=True, order=3)
        else:
            img = resize(img, (101, 101), preserve_range=True, order=3)
    return img

def my_crop_a(imgs, new_img_size=new_img_size):
    imgs = [my_crop(img, new_img_size) for img in imgs]
    return imgs

Detecting images with very small or very large masks

In [ ]:
def small_mask(mask, thr=0.005):
    res = np.mean(mask)
    return (res < thr) and (res > 0)

In [ ]:
def large_mask(mask, thr=0.997):
    res = np.mean(mask)
    return (res > thr) and (res < 1-thr)

Creation of down sample ground truth for deep supervision

In [ ]:
def get_down_masks(y1):
    y2 = [resize(m, (new_img_size//2, new_img_size//2), preserve_range=True, order=3) for m in (y1) ]
    y3 = [resize(m, (new_img_size//4, new_img_size//4), preserve_range=True, order=3) for m in (y1) ]
    y4 = [resize(m, (new_img_size//8, new_img_size//8), preserve_range=True, order=3) for m in (y1) ]
    y5 = [resize(m, (8, 8), preserve_range=True, order=3) for m in (y1) ]
    y2 = [(m > 0.5).astype('float32') for m in y2]
    y3 = [(m > 0.5).astype('float32') for m in y3]
    y4 = [(m > 0.5).astype('float32') for m in y4]
    y5 = [(m > 0.5).astype('float32') for m in y5]
    y2 = np.array(y2).reshape(-1, new_img_size//2, new_img_size//2, 1).astype('float32')
    y3 = np.array(y3).reshape(-1, new_img_size//4, new_img_size//4, 1).astype('float32')
    y4 = np.array(y4).reshape(-1, new_img_size//8, new_img_size//8, 1).astype('float32')
    y5 = np.array(y5).reshape(-1, 8, 8, 1).astype('float32')
    
    y6 = np.amax(y1, axis=(1,2), keepdims=True)
    
    return [y1, y2, y3, y4, y5, y6, y1]

Helper functions for tta continued.

In [ ]:
def get_train_data(x_train, y_train):
    x_train = [x for x,m in zip(x_train, y_train) if not vertical_mask(m)]
    y_train = [m for m in y_train if not vertical_mask(m)]
    
    x_train = [x for x,m in zip(x_train, y_train) if not small_mask(m)]
    y_train = [m for m in y_train if not small_mask(m)]

    x_train = [x for x,m in zip(x_train, y_train) if not large_mask(m)]
    y_train = [m for m in y_train if not large_mask(m)]

    x_train = np.append(x_train, [np.fliplr(x) for x in (x_train)], axis=0)
    y_train = np.append(y_train, [np.fliplr(x) for x in (y_train)], axis=0)

    x_train = np.array(x_train).reshape(-1, img_size, img_size, num_channels).astype('float32')
    y_train = np.array(y_train).reshape(-1, img_size, img_size, 1).astype('float32')
        
    return x_train, y_train

def get_valid_data(x_valid, y_valid):
    x_valid = np.append(x_valid, [np.fliplr(x) for x in (x_valid)], axis=0)
    y_valid = np.append(y_valid, [np.fliplr(x) for x in (y_valid)], axis=0)

    x_valid = [my_resize(x, is_mask=False) for x in (x_valid)]
    y_valid = [my_resize(y, is_mask=True) for y in (y_valid)]
 
    x_valid = np.array(x_valid).reshape(-1, new_img_size, new_img_size, num_channels).astype('float32')
    y_valid = np.array(y_valid).reshape(-1, new_img_size, new_img_size, 1).astype('float32')
    
    y_valid = get_down_masks(y_valid)

    return x_valid, y_valid

def get_valid_data_noflip(x_valid, y_valid):
    x_valid = [my_resize(x, is_mask=False) for x in (x_valid)]
    y_valid = [my_resize(y, is_mask=True) for y in (y_valid)]
 
    x_valid = np.array(x_valid).reshape(-1, new_img_size, new_img_size, num_channels).astype('float32')
    y_valid = np.array(y_valid).reshape(-1, new_img_size, new_img_size, 1).astype('float32')
    
    y_valid = get_down_masks(y_valid)

    return x_valid, y_valid

def get_test_data(test):
    x_test = [my_resize(x, is_mask=False) for x in (test)]
    x_test = np.array(x_test).reshape(-1, new_img_size, new_img_size, num_channels).astype('float32')
    return x_test


def tta_train_data(x_train, y_train):
    #Data augmentation

    v_aug1 = [tta(x, y) for x,y in zip((x_train), y_train)]
    x_train = [x for x,y in v_aug1]
    y_train = [y for x,y in v_aug1]

    x_train = [my_resize(x, is_mask=False) for x in (x_train)]
    y_train = [my_resize(x, is_mask=True) for x in (y_train)]

    x_train = np.array(x_train).reshape(-1, new_img_size, new_img_size, num_channels).astype('float32')
    y_train = np.array(y_train).reshape(-1, new_img_size, new_img_size, 1).astype('float32')

    y_train = get_down_masks(y_train)

    return x_train, y_train

Sequence for streaming upsampled data and for triauning time augmentation.  I should have created one for validation and for test data as well.

In [ ]:
class TGS_train_seq(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        p = np.random.permutation(x_set.shape[0])
        self.x, self.y = x_set[p], y_set[p]
        self.batch_size = batch_size
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return tta_train_data(batch_x, batch_y)
    

Fast metric computation.  Way faster than the one used by organizers.

In [ ]:
def get_iou_vector(A, B, new_img_size=new_img_size):
    batch_size = A.shape[0]
    metric = 0.0
    if new_img_size == 128 and A.shape[1] == new_img_size:
        A = A[:, 13:13+101, 13:13+101]
        B = B[:, 13:13+101, 13:13+101]
    if new_img_size == 224 and A.shape[1] == new_img_size:
        A = A[:, 11:11+202, 11:11+202]
        B = B[:, 11:11+202, 11:11+202]
    if new_img_size == 256 and A.shape[1] == new_img_size:
        A = A[:, 27:27+202, 27:27+202]
        B = B[:, 27:27+202, 27:27+202]
        
    for batch in range(batch_size):
        t, p = A[batch], B[batch]
        true = np.sum(t)
        pred = np.sum(p)
        if true == 0:
            metric += (pred == 0)
            continue
        
        intersection = np.sum(t * p)
        union = true + pred - intersection
        iou = intersection / union
        iou = min(1, np.floor(max(0, (iou - 0.45)*20)) / 10)
        
        metric += iou
    metric /= batch_size
    return metric


def iou_m(label, pred):
    return tf.py_func(get_iou_vector, [label, pred > 0.5], tf.float64)



Modified Focal loss.  Gamma = 1, and alpha computed per image.

https://arxiv.org/abs/1708.02002

In [ ]:
import keras.backend as K

epsilon = 0.1

def pixelwise_crossentropy(target, output):
    _epsilon = 10e-8
    output = K.sigmoid(output)
    output = tf.clip_by_value(output, _epsilon, 1. - _epsilon)
    cov =  tf.reduce_sum(target, [1, 2], keepdims=True) / tf.cast((tf.shape(target)[1]**2), 'float32')
    pos_weight = 0.5 / (cov + epsilon)
    neg_weight = 0.5 / (1 - cov + epsilon)
    
    return - tf.reduce_sum(target * pos_weight *  (1. - output) * tf.log(output) +
                           (1 - target)  *  neg_weight * output * tf.log(1 - output),
                           len(output.get_shape()) - 1)


Lovasz loss.

Simplified the code downloaded from: https://github.com/bermanmaxim/LovaszSoftmax


In [ ]:
def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    gts = tf.reduce_sum(gt_sorted)
    intersection = gts - tf.cumsum(gt_sorted)
    union = gts + tf.cumsum(1. - gt_sorted)
    jaccard = 1. - intersection / union
    jaccard = tf.concat((jaccard[0:1], jaccard[1:] - jaccard[:-1]), 0)
    return jaccard


# --------------------------- BINARY LOSSES ---------------------------

def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        def treat_image(log_lab):
            log, lab = log_lab
            log, lab = tf.expand_dims(log, 0), tf.expand_dims(lab, 0)
            log, lab = flatten_binary_scores(log, lab, ignore)
            return lovasz_hinge_flat(log, lab)
        losses = tf.map_fn(treat_image, (logits, labels), dtype=tf.float32)
        loss = tf.reduce_mean(losses)
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """

    def compute_loss():
        labelsf = tf.cast(labels, logits.dtype)
        signs = 2. * labelsf - 1.
        errors = 1. - logits * tf.stop_gradient(signs)
        errors_sorted, perm = tf.nn.top_k(errors, k=tf.shape(errors)[0], name="descending_sort")
        gt_sorted = tf.gather(labelsf, perm)
        grad = lovasz_grad(gt_sorted)
        loss = tf.tensordot(tf.nn.relu(errors_sorted), tf.stop_gradient(grad), 1, name="loss_non_void")
        return loss

    # deal with the void prediction case (only void pixels)
    loss = tf.cond(tf.equal(tf.shape(logits)[0], 0),
                   lambda: tf.reduce_sum(logits) * 0.,
                   compute_loss,
                   strict=True,
                   name="loss"
                   )
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = tf.reshape(scores, (-1,))
    labels = tf.reshape(labels, (-1,))
    if ignore is None:
        return scores, labels
    valid = tf.not_equal(labels, ignore)
    vscores = tf.boolean_mask(scores, valid, name='valid_scores')
    vlabels = tf.boolean_mask(labels, valid, name='valid_labels')
    return vscores, vlabels

def lovasz_loss(y_true, y_pred):
    y_true, y_pred = K.cast(K.squeeze(y_true, -1), 'int32'), K.cast(K.squeeze(y_pred, -1), 'float32')
    #logits = K.log(y_pred / (1. - y_pred))
    logits = y_pred #Jiaxin
    loss = lovasz_hinge(logits, y_true, per_image = True, ignore = None)
    return loss

Unet using pretrained resnet34 encoder from https://github.com/qubvel/segmentation_models

Modified for deep supervision and for empty mask classification.

In [ ]:
DEFAULT_DEEP_CONNECTIONS = {
    'resnet34':  ['decoder_stage%d_relu2' % i for i in range(3, 0, -1)] + ['relu1'],
    'resnext50': ['decoder_stage%d_relu2' % i for i in range(3, 0, -1)] + ['stage4_unit3_relu'],
}

def get_deep_connection(backbone):
    return DEFAULT_DEEP_CONNECTIONS[backbone]

In [ ]:
def get_layer_number(model, layer_name):
    """
    Help find layer in Keras model by name
    Args:
        model: Keras `Model`
        layer_name: str, name of layer
    Returns:
        index of layer
    Raises:
        ValueError: if model does not contains layer with such name
    """
    #print(layer_name)
    for i, l in enumerate(model.layers):
        if l.name == layer_name:
            return i
    raise ValueError('No layer with name {} in  model {}.'.format(layer_name, model.name))

def extract_outputs(model, layers, include_top=False):
    """
    Help extract intermediate layer outputs from model
    Args:
        model: Keras `Model`
        layer: list of integers/str, list of layers indexes or names to extract output
        include_top: bool, include final model layer output
    Returns:
        list of tensors (outputs)
    """
    
    #print(layers)
    layers_indexes = ([get_layer_number(model, l) if isinstance(l, str) else l
                      for l in layers])
    outputs = [model.layers[i].output for i in layers_indexes]

    if include_top:
        outputs.insert(0, model.output)
    #print(outputs)
    return outputs

In [ ]:
def build_model(backbone_name, input_shape):
    model = Unet(backbone_name=backbone_name, encoder_weights='imagenet',
                  decoder_block_type='transpose', activation=None, input_shape=input_shape,
                 )
    deep_connections = get_deep_connection(backbone_name)
    
    #print(deep_connections)

    deep_connection_outputs = extract_outputs(model, deep_connections) 

    output_layers_no_acti = [ Conv2D(1, (1,1), 
                                    padding="same", activation=None, 
                                    name='o_%i' % i)(output) for i,output in enumerate(deep_connection_outputs)]    
    output_layers_no_acti = [model.output] + output_layers_no_acti
    non_empty = AveragePooling2D((8, 8))(deep_connection_outputs[-1])
   
    output_non_empty_noActi = Dense(1, activation='linear', name='non_empty')(non_empty)
    
    full_output_noActi = Add(name='full')([model.output, output_non_empty_noActi])
    
    outputs_noActi = output_layers_no_acti + [output_non_empty_noActi, full_output_noActi]
    model_1 = Model(model.input, outputs_noActi)
    return model_1


7 fold training.  50 epochs with pixeclwise cross entropy, then 6 with Lovasz Loss for the final segmentation.

In [ ]:
kf = StratifiedKFold(7, shuffle=True, random_state=0)

In [ ]:
# Create train/validation split stratified by salt coverage
for fold, (train_idx, val_idx) in enumerate(kf.split(train, coverage)):
    print('***************************************************')
    print('*******************   fold  %d   *******************' % fold)
    print('***************************************************')

    basic_name = 'Unet_'+fname+'_fold_%d'%fold
    save_model_name = basic_name + '.model'
    print(save_model_name)
    try:
        sess.close()
    except:
        pass

    sess = init_seeds(0)
    
    x_train = train[train_idx]
    y_train = train_mask[train_idx]
    x_valid = train[val_idx]
    y_valid = train_mask[val_idx]
    x_train, y_train = get_train_data(x_train, y_train)
    x_valid, y_valid = get_valid_data(x_valid, y_valid)
    
    batch_size = 16
    train_seq = TGS_train_seq(x_train, y_train, batch_size=batch_size)

    model1 = build_model(backbone_name='resnet34', input_shape=(new_img_size, new_img_size, num_channels)
                 )
    if fold == 0:
        model1.summary()

    c = Adam(lr=0.001)
    weights = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 1.0]
    sum_weights = np.sum(weights)
    loss_weights = [w/sum_weights for w in weights]
    #loss_weights = [1.]
    model1.compile(loss=pixelwise_crossentropy, optimizer=c, metrics=[iou_m],
                   loss_weights=loss_weights)


    model_checkpoint = ModelCheckpoint(save_model_name, monitor='val_full_iou_m', save_weights_only=True,
                                       mode = 'max', save_best_only=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_full_iou_m', mode = 'max',factor=0.5, 
                                  patience=5, min_lr=0.0001, verbose=1)

    epochs = 50
    history = model1.fit_generator(train_seq,
                                    validation_data=[x_valid, y_valid], 
                                    epochs=epochs,
                                    callbacks=[ model_checkpoint, reduce_lr], 
                                    verbose=1,
                                  shuffle=True)

    fig, (ax_loss, ax_score) = plt.subplots(1, 2, figsize=(15,5))
    ax_loss.plot(history.epoch, history.history["full_loss"], label="Train loss")
    ax_loss.plot(history.epoch, history.history["val_full_loss"], label="Validation loss")
    ax_loss.legend()
    ax_score.plot(history.epoch, history.history["full_iou_m"], label="Train score")
    ax_score.plot(history.epoch, history.history["val_full_iou_m"], label="Validation score")
    ax_score.legend()

    save_model_name2 = basic_name + '_2.model'

    print(save_model_name2)


    try:
        sess.close()
    except:
        pass

    sess = init_seeds(0)

    model1 = build_model(backbone_name='resnet34', input_shape=(new_img_size, new_img_size, num_channels))
    model1.load_weights(save_model_name)
 
    c = Adam(lr = 0.0005, clipvalue=1)
    loss = [pixelwise_crossentropy, pixelwise_crossentropy, pixelwise_crossentropy, 
           pixelwise_crossentropy, pixelwise_crossentropy, pixelwise_crossentropy,
            lovasz_loss
           ]
    model1.compile(loss=loss, optimizer=c, metrics=[iou_m],
                   loss_weights=loss_weights)
    #model1.summary()

    model_checkpoint = ModelCheckpoint(save_model_name2, monitor='val_full_iou_m', save_weights_only=True, 
                                       mode = 'max', save_best_only=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_full_iou_m', mode = 'max',factor=0.5, 
                                  patience=5, min_lr=0.0001, verbose=1)

    epochs = 36
    batch_size = 16
    history = model1.fit_generator(train_seq,
                                    validation_data=[x_valid, y_valid], 
                                    epochs=epochs,
                                    callbacks=[ model_checkpoint, reduce_lr], 
                                    verbose=1,
                                  shuffle=True)

    fig, (ax_loss, ax_score) = plt.subplots(1, 2, figsize=(15,5))
    ax_loss.plot(history.epoch, history.history["full_loss"], label="Train loss")
    ax_loss.plot(history.epoch, history.history["val_full_loss"], label="Validation loss")
    ax_loss.legend()
    ax_score.plot(history.epoch, history.history["full_iou_m"], label="Train score")
    ax_score.plot(history.epoch, history.history["val_full_iou_m"], label="Validation score")
    ax_score.legend()
    plt.show()

Helper functions for best model prediction

In [ ]:
def predict_result(model, x_test, img_size): # predict both orginal and reflect x
    x_test_reflect = [my_crop(x) for x in x_test]
    x_test_reflect = [np.fliplr(x) for x in x_test_reflect]
    x_test_reflect = [my_resize(x, is_mask=False) for x in x_test_reflect]
    x_test_reflect = np.array(x_test_reflect)
    preds_test = model.predict(x_test)[-1].reshape(-1, new_img_size, new_img_size)
    preds_test2_refect = model.predict(x_test_reflect)[-1].reshape(-1, new_img_size, new_img_size)
    preds_test = my_crop_a(preds_test)
    preds_test2_refect = my_crop_a(preds_test2_refect)
    preds_test2 = np.array([ np.fliplr(x) for x in preds_test2_refect] )
    return preds_test, preds_test2

In [ ]:
def iou_coef(y_true, y_pred, threshold, verbose=False, ret_all=False):
    y_true = y_true.flatten() 
    y_pred = 1 * (y_pred.flatten() > threshold)
    inter = y_true * y_pred
    union = y_true + y_pred - inter
    true_size = np.sum(y_true)
    pred_size = np.sum(y_pred)
    inter_size = np.sum(inter)
    union_size = np.sum(union)
    iou = np.sum(inter) / np.sum(union)
    if verbose:
        print(true_size, pred_size, inter_size, union_size)
    if true_size == 0:
        if ret_all:
            return 1 * (pred_size == 0), true_size, pred_size, inter_size, union_size
        return 1 * (pred_size == 0)
    iou = min(1, np.floor(max(0, (iou - 0.45)*20)) / 10)
    if ret_all:
        return iou, true_size, pred_size, inter_size, union_size
    return  iou

def iou_mean(thr, y_valid, pred_valid):
    data=([iou_coef(y_valid[idx], pred_valid[idx], thr, verbose=False, ret_all=False) \
                        for idx in range(len(y_valid))])
    return np.mean(data)

Reading test data

In [ ]:
test = []
for n, id_ in tqdm_notebook(enumerate(test_ids)):
    file = "../input/test/images/{}.png".format(id_)
    image = (imread(file) / 255).astype('float32')
    #image = rescale_intensity(image, out_range=np.uint8).astype('float32') / 255
    test.append(image)
    
x_test = get_test_data(test)
x_test.shape

In [ ]:
"""
used for converting the decoded image to rle mask
Fast compared to previous one
"""
def rle_encode(im):
    '''
    im: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

Compute score for each fold.  

Combpute best threshold (not used for final submission).

In [ ]:
y_valids = []
valid_preds = []

kf = StratifiedKFold(7, shuffle=True, random_state=0)

for fold, (train_idx, val_idx) in enumerate(kf.split(train, coverage)):
        
    print('*******************   fold  %d   *******************' % fold)

    basic_name = 'Unet_'+fname+'_fold_%d'%fold
    
    save_model_name2 = basic_name + '_2.model'

    print(save_model_name2)
    
    x_valid = train[val_idx]
    y_valid = train_mask[val_idx]
    x_valid, y_valid = get_valid_data_noflip(x_valid, y_valid)

    y_valid = train_mask[val_idx]
    

    try:
        sess.close()
    except:
        pass

    sess = init_seeds(0)

    model1 = build_model(backbone_name='resnet34', input_shape=(new_img_size, new_img_size, num_channels))
    model1.load_weights(save_model_name2)


    pred_valid, pred_valid2 = predict_result(model1, x_valid, img_size)


    pred1 = expit((pred_valid + pred_valid2) / 2)
    
    
    ious = [iou_mean(thr, y_valid, pred1) for thr in np.linspace(0, 1, 101)]


    threshold_best = np.argmax(ious)
    
    print(threshold_best/100, ious[threshold_best])
    y_valids.append(y_valid)
    valid_preds.append(pred1)
    


Compute test prediction and submission

In [ ]:
test_preds = np.zeros((18000, 101, 101))

kf = StratifiedKFold(7, shuffle=True, random_state=0)

for fold, (train_idx, val_idx) in enumerate(kf.split(train, coverage)):
        
    print('*******************   fold  %d   *******************' % fold)

    basic_name = 'Unet_'+fname+'_fold_%d'%fold
    
    save_model_name2 = basic_name + '_2.model'

    print(save_model_name2)
    
    try:
        sess.close()
    except:
        pass

    sess = init_seeds(0)

    model1 = build_model(backbone_name='resnet34', input_shape=(new_img_size, new_img_size, num_channels))
    model1.load_weights(save_model_name2)

    for i in tqdm_notebook(range(0, 18000, 1000)):
        pred_test, pred_test1 = predict_result(model1, x_test[i:i+1000], img_size)

        pred2 = (pred_test + pred_test1) / 2
        test_preds[i:i+1000] += expit(pred2)
test_preds /= 7

In [ ]:
submission_file = '../submissions/'+fname+'_all_expit.csv'

print(submission_file)

pred_dict = {idx: rle_encode(test_preds[i] > 0.5) for i, idx in enumerate(tqdm_notebook(test_ids))}


sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv(submission_file)


sub

Save fold and test raw prediciton for ensembling

In [ ]:
all_valid_preds = np.concatenate(valid_preds)


import pickle as pkl

with open('../data/%s_all_valid_preds_all.pkl' % fname, 'wb') as file:
    pkl.dump(all_valid_preds, file)   

with open('../data/%s_pred_test_all.pkl' % fname, 'wb') as file:
    pkl.dump(test_preds, file)
